In [1]:
import sys
import os
sys.path.append("..")
from knool.satellite.data.satellite import TLEsPerSat
from knool.geodata import geo_info,geo_geom,geo_io
import pandas as pd
from skyfield.api import load
from datetime import datetime as dt
from datetime import timedelta
from skyfield.api import utc
import numpy as np
from osgeo import gdal, osr, ogr
import importlib

In [2]:
#HOW_TO_USE_SKYFIELD
##READ https://qiita.com/grinpeaceman/items/22f06714a98dbfacce88

tles = load.tle_file(r"../knool/satellite/data/data/two_line_elements/sat40697.txt") # TLE読み込み
tles = load.tle_file(r"../knool/satellite/data/data/two_line_elements/sat38337.txt") # TLE読み込み
satellite=tles[1354]
sgp4_model = satellite.model

print("衛星名: ", satellite.name)
print("元期: ", satellite.epoch.utc)#_jpl())
print("衛星番号: ", sgp4_model.satnum)
print("エポック年: ", sgp4_model.epochyr)
print("エポック日: ", sgp4_model.epochdays)
print("エポックのユリウス日: ", sgp4_model.jdsatepoch)
print("ndot: ", sgp4_model.ndot)
print("nddot: ", sgp4_model.nddot)
print("弾道抗力係数 B*: ", sgp4_model.bstar)
print("軌道傾斜角[degree]: ", sgp4_model.inclo*180/3.14)
print("昇交点赤経[degree]: ", sgp4_model.nodeo*180/3.14)
print("離心率: ", sgp4_model.ecco)
print("近地点引数[rad]: ", sgp4_model.argpo)
print("平均近点角[rad]: ", sgp4_model.mo)
print("平均運動[rad/min]: ", sgp4_model.no_kozai)

衛星名:  None
元期:  CalendarTuple(year=2015, month=9, day=21, hour=22, minute=19, second=19.809722900390625)
衛星番号:  38337
エポック年:  15
エポック日:  264.93009039
エポックのユリウス日:  2457286.5
ndot:  6.7570906804641574e-12
nddot:  0.0
弾道抗力係数 B*:  5.9498e-05
軌道傾斜角[degree]:  98.25140927731319
昇交点赤経[degree]:  203.37620306765075
離心率:  0.0001257
近地点引数[rad]:  1.1657612579430765
平均近点角[rad]:  5.846975167228644
平均運動[rad/min]:  0.06357998806296415


In [3]:
#HOW_TO_USE_SKYFIELD
##READ https://qiita.com/grinpeaceman/items/22f06714a98dbfacce88

#get lat,lon,elevation from a tle
ts = load.timescale()  # skyfiled上で時刻を扱うモジュール
tle=tles[0]
date1 = dt(2018, 2, 1, 12, 15, 30, 2000, tzinfo=utc)
date1_sf=ts.from_datetime(date1)

geocentric=tle.at(date1_sf) #get xyz
subpoint = geocentric.subpoint()
lat = subpoint.latitude.degrees
lon = subpoint.longitude.degrees
ele = subpoint.elevation.m
print(geocentric.velocity)
print(geocentric.position)
print(geocentric.xyz)
print(lat,lon,ele)

[-0.00358946  0.00198662  0.00146756] au/day
[ 1.02781862e-05 -1.37594739e-05  4.37801473e-05] au
[ 1.02781862e-05 -1.37594739e-05  4.37801473e-05] au
68.75920171845111 -8.743172725379656 675763.1245518889


In [4]:
#convert lla/ecef from ecef/lla using our package.
print(lat,lon,ele)
x,y,z=geo_info.transform_lla_to_ecef(lat,lon,ele)
print(x,y,z) # meter
lat,lon,ele=geo_info.transform_ecef_to_lla(x,y,z)
print(lat,lon,ele)

68.75920171845111 -8.743172725379656 675763.1245518889
2532519.7660571965 -389483.15910038026 6552150.204388009
68.75920172679525 -8.74317272537966 675763.1271996461


In [7]:
#create a kml file of a orbit line
infile=r"../knool/satellite/data/data/two_line_elements/sat38337.txt"
test=TLEsPerSat(infile)

sdate=dt.strptime("2015-09-21 00:00:00",'%Y-%m-%d %H:%M:%S')
edate=dt.strptime("2015-09-23 11:40:00",'%Y-%m-%d %H:%M:%S')
test.set_period(sdate, edate) # for faster computation
# test.calc_positions_between("2015-09-22 00:58:00","2015-09-22 01:40:00",10)

sdate=dt.strptime("2015-09-22 00:58:00",'%Y-%m-%d %H:%M:%S')
edate=dt.strptime("2015-09-22 01:40:00",'%Y-%m-%d %H:%M:%S')
test.calc_positions_between(sdate,edate,3)
test.export("../../test_data/output/test.kml")

# fdate=sdate
# ldate=edate
# interval=10
# ds_date = pd.date_range(start=fdate, end=ldate,freq=str(interval) + "min", tz=utc)
# df=test.df
# df
# for date in ds_date:
#     print(date)
#     print(df.index.unique().get_indexer([pd.to_datetime(date)], method="nearest"))
#     row1 = df.loc[df.index == df.index.unique()[0]]


2015-09-22 00:58:00+00:00
2015-09-22 01:01:00+00:00
2015-09-22 01:04:00+00:00
2015-09-22 01:07:00+00:00
2015-09-22 01:10:00+00:00
2015-09-22 01:13:00+00:00
2015-09-22 01:16:00+00:00
2015-09-22 01:19:00+00:00
2015-09-22 01:22:00+00:00
2015-09-22 01:25:00+00:00
2015-09-22 01:28:00+00:00
2015-09-22 01:31:00+00:00
2015-09-22 01:34:00+00:00
2015-09-22 01:37:00+00:00
2015-09-22 01:40:00+00:00


In [9]:
#create a kml file of a orbit line
test.calc_positions_faster_between(sdate,edate,1)
test.export("../../test_data/output/test2.kml")

TypeError: DatetimeIndex.get_loc() got an unexpected keyword argument 'method'

In [10]:
#Creates a kml file of a polygon
test.calc_buff_area_between(sdate,edate,1,1000000,ori="right")
test.export("../../test_data/output/test3.kml")

TypeError: DatetimeIndex.get_loc() got an unexpected keyword argument 'method'

In [11]:
sdate=dt.strptime("2011-07-01 10:00:00",'%Y-%m-%d %H:%M:%S')
edate=dt.strptime("2011-07-01 11:40:00",'%Y-%m-%d %H:%M:%S')
test.calc_scene_areas(sdate,edate,1,1450000,6,ori="middle")
test.export("../../test_data/output/test4.kml")

TypeError: DatetimeIndex.get_loc() got an unexpected keyword argument 'method'

In [12]:
from knool.satellite.data import satellite
importlib.reload(geo_info)
importlib.reload(geo_geom)
importlib.reload(geo_io)
importlib.reload(satellite)


shpfile=r"../../test_data/matchup/AOI2.shp"
vector=ogr.Open(shpfile)
layer=vector.GetLayer()
geom_list=[]
for feat in layer:
    geom_list.append(feat.GetGeometryRef().Clone())

trans_for=[]
trans_back=[]
for geom in geom_list:
    lon,lat,_=geom.Centroid().GetPoint()
    srs_pre=geo_info.get_shifted_wgs84_proj4(0)
    srs_af=geo_info.get_orthographic_proj4(lat,lon,0,0)
    trans_for.append(geo_info.get_coord_transform_proj4(srs_pre,srs_af))
    trans_back.append(geo_info.get_coord_transform_proj4(srs_af,srs_pre))

sdate=dt.strptime("2011-06-01 03:00:00",'%Y-%m-%d %H:%M:%S')
edate=dt.strptime("2011-06-02 05:00:00",'%Y-%m-%d %H:%M:%S')

infile=r"../knool/satellite/data/data/two_line_elements/sat38337.txt"
test=satellite.TLEsPerSat(infile)


In [13]:
# shpfile=r"../../test_data/matchup/AOI.shp"
# vector=ogr.Open(shpfile)
# layer=vector.GetLayer()
# geom_list=[]
# for feat in layer:
#     geom_list.append(feat.GetGeometryRef().Clone())

# trans_list=[]
# for geom in geom_list:
#     lon,lat,_=geom.Centroid().GetPoint()
#     srs_pre=geo_info.get_shifted_wgs84_proj4(0)
#     srs_af=geo_info.get_orthographic_proj4(lat,lon,0,0)
#     trans_fore=geo_info.get_coord_transform_proj4(srs_pre,srs_af)
#     trans_back=geo_info.get_coord_transform_proj4(srs_af,srs_pre)
#     trans_list.append([trans_fore,trans_back])

sdate=dt.strptime("2011-06-01 03:00:00",'%Y-%m-%d %H:%M:%S')
edate=dt.strptime("2011-06-02 05:00:00",'%Y-%m-%d %H:%M:%S')
test.calc_scene_areas(sdate,edate,1,2330000,2,ori="middle")
test.export("../../test_data/output/test4.kml")

result,result2 = test.calc_intersect_scene_areas(sdate,edate,1,2330000,2,geom_list,trans_for,ori="middle")
result=test.transform(result,trans_back)
#print(len(result[0]),result)
test.export(r"../../test_data/output/test5.shp",option=0)
result=test.transform(result2,trans_back)
test.export(r"../../test_data/output/test6.kml",option=0)


TypeError: DatetimeIndex.get_loc() got an unexpected keyword argument 'method'

In [14]:
sensor_name="amsr2"
sensor1=satellite.Sensor(sensor_name)
# sensor_name="modis"
# sensor2=satellite.Sensor(sensor_name)
print(sensor1.ncid)
sdate=dt.strptime("2011-06-01 03:00:00",'%Y-%m-%d %H:%M:%S')
edate=dt.strptime("2011-06-01 03:30:00",'%Y-%m-%d %H:%M:%S')

geom_s1_llist00,_=sensor1.calc_intersect_scene_areas(sdate,edate,1,2,geom_list,trans_for,scene_all=True)
print(len(geom_s1_llist00[0]))
geom_s1_llist0,attr_s1_dlist=sensor1.union_adjacent_scenes(geom_s1_llist00)
print(geom_s1_llist0)
geom_s1_llist=sensor1.transform(geom_s1_llist0,trans_back)
print(geom_s1_llist)
#sensor1.export(r"../../test_data/output/test7.shp",option=0)
# sensor1.export(r"../../test_data/output/final2.shp",option=0)

TypeError: initialize_config_dir.__init__() got an unexpected keyword argument 'version_base'

SyntaxError: expected 'except' or 'finally' block (4168339267.py, line 3)